In [1]:
!pip install datasets -qq #สำหรับโหลดdataset
!pip install transformers -qq #สำหรับเทรนโมเดล
!pip install seqeval -qq #สำหรับevaluation

     |████████████████████████████████| 311 kB 5.2 MB/s 
     |████████████████████████████████| 243 kB 9.9 MB/s 
     |████████████████████████████████| 133 kB 13.0 MB/s 
     |████████████████████████████████| 67 kB 2.4 MB/s 
     |████████████████████████████████| 1.1 MB 17.1 MB/s 
     |████████████████████████████████| 271 kB 30.0 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
     |████████████████████████████████| 144 kB 28.4 MB/s 
     |████████████████████████████████| 3.5 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 44.8 MB/s 
     |████████████████████████████████| 6.8 MB 32.3 MB/s 
     |████████████████████████████████| 895 kB 44.6 MB/s 
     |████████████████████████████████| 43 kB 1.2 MB/s 


In [2]:
### download LST20 dataset
!gdown --id 148vJ_8Tjoh-HZAuHBg_a5x0ebveoL_Mf
!tar -xvf /content/AIFORTHAI-LST20Corpus.tar.gz

Streaming output truncated to the last 5000 lines.
LST20_Corpus/train/._T12371.txt
LST20_Corpus/train/T12371.txt
LST20_Corpus/train/._T11678.txt
LST20_Corpus/train/T11678.txt
LST20_Corpus/train/._T10566.txt
LST20_Corpus/train/T10566.txt
LST20_Corpus/train/._T00775.txt
LST20_Corpus/train/T00775.txt
LST20_Corpus/train/._T12417.txt
LST20_Corpus/train/T12417.txt
LST20_Corpus/train/._T12403.txt
LST20_Corpus/train/T12403.txt
LST20_Corpus/train/._T10214.txt
LST20_Corpus/train/T10214.txt
LST20_Corpus/train/._T01319.txt
LST20_Corpus/train/T01319.txt
LST20_Corpus/train/._T00761.txt
LST20_Corpus/train/T00761.txt
LST20_Corpus/train/._T01457.txt
LST20_Corpus/train/T01457.txt
LST20_Corpus/train/._T00991.txt
LST20_Corpus/train/T00991.txt
LST20_Corpus/train/._T11644.txt
LST20_Corpus/train/T11644.txt
LST20_Corpus/train/._T00749.txt
LST20_Corpus/train/T00749.txt
LST20_Corpus/train/._T13053.txt
LST20_Corpus/train/T13053.txt
LST20_Corpus/train/._T11122.txt
LST20_Corpus/train/T11122.txt
LST20_Corpus/train/

In [3]:
import numpy as np
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification

In [4]:
task = "ner" #อันนี้เราset taskname เป็น ner
model_checkpoint = "Geotrend/bert-base-th-cased" #หลักๆเราจะใช้ language model ของ Geotrend ภาษาไทย 
batch_size = 16 #gpu เราบน colab น่าจะได้ batchsize แค่16
datasets = load_dataset("lst20", data_dir="/content/LST20_Corpus") #โหลดdataset

Downloading:   0%|          | 0.00/2.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

Using custom data configuration default-97621787f5b60e1b


0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset lst20 downloaded and prepared to /root/.cache/huggingface/datasets/lst20/default-97621787f5b60e1b/0.0.0/e1b2a921fb011578ab43ddbbf789f3c500d62cb2df8ae4ed4b60bae8e4c0d3ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
datasets
#train set = 63310, validate set = 5620, test set = 5250

DatasetDict({
    train: Dataset({
        features: ['id', 'fname', 'tokens', 'pos_tags', 'ner_tags', 'clause_tags'],
        num_rows: 63310
    })
    validation: Dataset({
        features: ['id', 'fname', 'tokens', 'pos_tags', 'ner_tags', 'clause_tags'],
        num_rows: 5620
    })
    test: Dataset({
        features: ['id', 'fname', 'tokens', 'pos_tags', 'ner_tags', 'clause_tags'],
        num_rows: 5250
    })
})

In [6]:
datasets['train'][0]

{'clause_tags': [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3],
 'fname': 'T00126.txt',
 'id': '0',
 'ner_tags': [8, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0],
 'pos_tags': [0, 1, 1, 1, 2, 0, 2, 4, 0, 12, 1, 0],
 'tokens': ['สุรยุทธ์',
  'ยัน',
  'ปฏิเสธ',
  'ลงนาม',
  '_',
  'MOU',
  '_',
  'กับ',
  'อียู',
  'ไม่',
  'กระทบ',
  'สัมพันธ์']}

In [7]:
datasets['train'].features['ner_tags']

Sequence(feature=ClassLabel(num_classes=31, names=['O', 'B_BRN', 'B_DES', 'B_DTM', 'B_LOC', 'B_MEA', 'B_NUM', 'B_ORG', 'B_PER', 'B_TRM', 'B_TTL', 'I_BRN', 'I_DES', 'I_DTM', 'I_LOC', 'I_MEA', 'I_NUM', 'I_ORG', 'I_PER', 'I_TRM', 'I_TTL', 'E_BRN', 'E_DES', 'E_DTM', 'E_LOC', 'E_MEA', 'E_NUM', 'E_ORG', 'E_PER', 'E_TRM', 'E_TTL'], names_file=None, id=None), length=-1, id=None)

In [8]:
datasets['train'].features['ner_tags'].feature.names

['O',
 'B_BRN',
 'B_DES',
 'B_DTM',
 'B_LOC',
 'B_MEA',
 'B_NUM',
 'B_ORG',
 'B_PER',
 'B_TRM',
 'B_TTL',
 'I_BRN',
 'I_DES',
 'I_DTM',
 'I_LOC',
 'I_MEA',
 'I_NUM',
 'I_ORG',
 'I_PER',
 'I_TRM',
 'I_TTL',
 'E_BRN',
 'E_DES',
 'E_DTM',
 'E_LOC',
 'E_MEA',
 'E_NUM',
 'E_ORG',
 'E_PER',
 'E_TRM',
 'E_TTL']

In [9]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names

In [10]:
label_list

['O',
 'B_BRN',
 'B_DES',
 'B_DTM',
 'B_LOC',
 'B_MEA',
 'B_NUM',
 'B_ORG',
 'B_PER',
 'B_TRM',
 'B_TTL',
 'I_BRN',
 'I_DES',
 'I_DTM',
 'I_LOC',
 'I_MEA',
 'I_NUM',
 'I_ORG',
 'I_PER',
 'I_TRM',
 'I_TTL',
 'E_BRN',
 'E_DES',
 'E_DTM',
 'E_LOC',
 'E_MEA',
 'E_NUM',
 'E_ORG',
 'E_PER',
 'E_TRM',
 'E_TTL']

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/658 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/46.6k [00:00<?, ?B/s]

tokenizer จะช่วยในการตัด sequence ตั้งแต่ระดับวลี ไปถึงระดับตัวอักษร ขึ้นกับว่า pretrained ตัวนี้เทรนมาแบบไหน จากเท่าที่ดู เหมือนว่า “Geotrend/bert-base-th-cased” จะถูกเทรนมาแบบ multi-language ทำให้ tokenizer อาจมีระดับย่อยถึง 1 character งั้นมาลองดูกัน

In [12]:
print(tokenizer("ผมเป็นคนไทย")["input_ids"])
print(tokenizer(["ผม","เป็น","คนไทย"], is_split_into_words=True)["input_ids"])

[11, 162, 2939, 6762, 4949, 2617, 5901, 12]
[11, 162, 2939, 7439, 4344, 5901, 12]


In [13]:
sent1 = tokenizer("ผมเป็นคนไทย")
sent2 = tokenizer(["ผม","เป็น","คนไทย"], is_split_into_words=True)
print(tokenizer.convert_ids_to_tokens(sent1["input_ids"]),
      tokenizer.convert_ids_to_tokens(sent2["input_ids"]))

['[CLS]', 'ผ', '##ม', '##เป็น', '##ค', '##น', '##ไทย', '[SEP]'] ['[CLS]', 'ผ', '##ม', 'เป็น', 'คน', '##ไทย', '[SEP]']


In [14]:
print(datasets["train"][5])

{'id': '5', 'fname': 'T00126.txt', 'tokens': ['(', '1', '_', 'ก.ย.', ')', '_', 'พล.อ.', 'สุรยุทธ์', '_', 'จุลานนท์', '_', 'กล่าว', 'รายการ', 'เปิด', 'บ้านพิษณุโลก', 'ออกอากาศ', 'ทาง', 'สถานี', 'วิทยุ', 'โทรทัศน์', 'แห่ง', 'ประเทศ', 'ไทย', 'ช่อง', '_', '11', '_', 'กรม', 'ประชาสัมพันธ์'], 'pos_tags': [2, 8, 2, 0, 2, 2, 0, 0, 2, 0, 2, 1, 0, 1, 0, 1, 0, 0, 0, 0, 4, 0, 0, 0, 2, 8, 2, 0, 1], 'ner_tags': [0, 3, 13, 23, 0, 0, 2, 8, 18, 28, 0, 0, 0, 0, 4, 0, 0, 7, 17, 17, 17, 17, 17, 17, 17, 27, 0, 7, 27], 'clause_tags': [1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3]}


In [15]:
# Aligning Labels
example = datasets["train"][5]
tokenized_input = tokenizer(example["tokens"], is_split_into_words =True)
print(len(tokenized_input["input_ids"]), len(example["ner_tags"]))

100 29


In [16]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    label_all_tokens = True
    labels = []

    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [17]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/64 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

Fine-tuning Model

ก่อนอื่นเราต้อง load model มาก่อน โดยโมเดลที่เราใช้คือ “Geotrend/bert-base-th-cased” หรือ BERT model นั้นเอง ความง่ายของ AutoModel ของ huggingface คือเราสามารถ deploy ได้เลยไม่ยุ่งยาก

In [18]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Downloading:   0%|          | 0.00/353M [00:00<?, ?B/s]

Some weights of the model checkpoint at Geotrend/bert-base-th-cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at Geotrend/bert-base-th-cased a

ของที่เราต้องใช้ ในการ fine-tuning model หลักๆเราต้องมี 3 functions ได้แก่ TrainingArguments, DataCollator, ComputationMetrics

เริ่มจาก TrainingArguments ก่อน TrainingArguments คือ การกำหนด parameters สำหรับ fine-tuning model เราสามารถกำหนด TrainingArguments ได้ดังต่อไปนี้

In [19]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

TrainingArguments สำคัญมากครับ โมเดลเราจะ converge เร็ว converge ช้าขึ้นอยู่ parameters และ มันสามารถปรับเพิ่มอะไรได้อีกเยอะ แน่ะนำให้ไปตำต่อที่นี้ครับ

Data Collator

Data Collator เป็น function ที่ทำให้ covert dataset จาก list ให้เป็น batch tensors ครับ สามารถเรียกใช้ได้ง่ายๆต่อไปนี้

In [20]:
data_collator = DataCollatorForTokenClassification(tokenizer)

Computation Metrics

อันนี้เอามาวัดผลของการ fine-tuning เราใช้ seqeval ซึ่งเป็น libary หนึ่งที่นิยมนำมาใช้กับการวัดผล งานพวก sequence tagging ครับ

ก่อนอื่นทำการโหลด seqeval ก่อน

In [21]:
metric = load_metric("seqeval")

Downloading:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

In [22]:
labels = [label_list[i].replace("_", "-") for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'DES': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'DTM': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'LOC': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'ORG': {'f1': 1.0, 'number': 2, 'precision': 1.0, 'recall': 1.0},
 'PER': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'overall_accuracy': 1.0,
 'overall_f1': 1.0,
 'overall_precision': 1.0,
 'overall_recall': 1.0}

In [23]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [label_list[p].replace("_", "-") for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l].replace("_", "-") for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

หลังจากเรามี functions ทั้งหมดแล้วก็พร้อมที่จะ fine-tuning model แล้วครับ โดย function ที่เราจะใช้คือ Trainerครับ

In [24]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

เราสามารถเริ่มเทรนได้ด้วยการ call trainer.train() ในการ train ค่อนข้างใช้เวลานานมากๆ 1 epoch ประมาณ 30–40 นาทีครับ

In [25]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: pos_tags, clause_tags, fname, tokens, ner_tags, id.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 63310
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 11871


Epoch,Training Loss,Validation Loss


RuntimeError: ignored

เราสามารถ evaluate model ได้ด้วยคำสั่ง

In [ ]:
trainer.evaluate()

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

true_predictions = [
    [label_list[p].replace("_", "-") for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l].replace("_", "-") for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
print(results)